In [ ]:
# Set up widgets for job parameters
dbutils.widgets.text("pbfPath", "", "Path to PBF file")
dbutils.widgets.text("geometryType", "WKT", "Geometry Type")
dbutils.widgets.text("emptyTagFilter", "true", "Empty Tag Filter (true/false)")
dbutils.widgets.text("keyFilter", "", "Key Filter (optional)")
dbutils.widgets.text("tagFilter", "", "Tag Filter (optional)")
dbutils.widgets.text("bronzeTable", "delta_table", "Output Delta Table")

In [ ]:
# Retrieve widget values
pbfPath = dbutils.widgets.get("pbfPath")
geometryType = dbutils.widgets.get("geometryType")
emptyTagFilter = dbutils.widgets.get("emptyTagFilter").lower() == "true"
keyFilter = dbutils.widgets.get("keyFilter") or None  # Set to None if empty
tagFilter = dbutils.widgets.get("tagFilter") or None  # Set to None if empty
bronzeTable = dbutils.widgets.get("bronzeTable")

In [ ]:
%run "./load_datasource"

In [ ]:
# Validate required parameters
if not pbfPath:
    raise ValueError("The 'pbfPath' parameter is required.")

# Build the reader options dynamically
reader = spark.read.format("pbf").option("path", pbfPath).option("geometryType", geometryType)

if emptyTagFilter:
    reader = reader.option("emptyTagFilter", True)

if keyFilter is not None:
    reader = reader.option("keyFilter", keyFilter)

if tagFilter is not None:
    reader = reader.option("tagFilter", tagFilter)

In [ ]:
# Load the data
df = reader.load()

In [ ]:
# Save the data to a Delta table
df.write.format("delta").mode("overwrite").saveAsTable(bronzeTable)

print(f"Data has been saved to the Delta table: {bronzeTable}")